Übung: kubectl-CLI und Basis Ressourcen
-------------------------------------------------------------

Das `kubectl`-Kommando stellt, eine der Schaltzentralen des K8s Clusters zur Administration der Ressourcen dar.

In dieser Übung verwenden wir das `kubectl`-Kommando zur Erstellen eines Pods und Services.

Das passiert in einer eigenen Namespace um die Resultate gezielt Darstellen zu können:

In [37]:
! kubectl create namespace test

namespace/test created


Erzeugen eines Pod's, hier der Apache Web Server.

Die Option `--restart=Never` erzeugt nur einen Pod. Ansonsten wird ein Deployment erzeugt.

In [38]:
! kubectl run apache --image=httpd --restart=Never --namespace test

pod/apache created


Ausgabe der Erzeugten Ergebnisse und die YAML Datei welche den Pod beschreibt:

In [39]:
! kubectl get pods,services --namespace test

NAME     READY   STATUS              RESTARTS   AGE
apache   0/1     ContainerCreating   0          2s


In [40]:
! kubectl get pod apache -o yaml --namespace test | head -19

apiVersion: v1
kind: Pod
metadata:
  creationTimestamp: 2018-10-25T08:47:03Z
  labels:
    run: apache
  name: apache
  namespace: test
  resourceVersion: "830513"
  selfLink: /api/v1/namespaces/test/pods/apache
  uid: 8bb19949-d832-11e8-9739-02fba84c29d0
spec:
  containers:
  - image: httpd
    imagePullPolicy: Always
    name: apache
    resources: {}
    terminationMessagePath: /dev/termination-log
    terminationMessagePolicy: File


Zu dem Pod `apache` Erzeugen wir einen Service. Dadurch wird der Web Server von aussen sichtbar.

Der Port 80 wird von Kubernetes automatisch auf den nächsten freien Port gemappt.

In [41]:
! kubectl expose pod/apache --type="LoadBalancer" --port 80 --namespace test

service/apache exposed


In [42]:
! kubectl get service apache -o yaml --namespace test | head -29

apiVersion: v1
kind: Service
metadata:
  creationTimestamp: 2018-10-25T08:47:36Z
  labels:
    run: apache
  name: apache
  namespace: test
  resourceVersion: "830551"
  selfLink: /api/v1/namespaces/test/services/apache
  uid: 9f395faa-d832-11e8-9739-02fba84c29d0
spec:
  clusterIP: 10.111.144.164
  externalTrafficPolicy: Cluster
  ports:
  - nodePort: 32243
    port: 80
    protocol: TCP
    targetPort: 80
  selector:
    run: apache
  sessionAffinity: None
  type: LoadBalancer
status:
  loadBalancer: {}


Wir sollten jetzt einen Pod und einen Service apache haben

In [43]:
! kubectl get pods,service apache --namespace test

NAME         READY   STATUS    RESTARTS   AGE
pod/apache   1/1     Running   0          43s

NAME             TYPE           CLUSTER-IP       EXTERNAL-IP   PORT(S)        AGE
service/apache   LoadBalancer   10.111.144.164   <pending>     80:32243/TCP   10s


Da wir keinen LoadBalancer haben müssen wir mit einem kleinen Shellscript selber die IP des Clusters und der gemappte Port als URL aufbereiten.

Diese Shellscript ist im Script `startsvc` hinterlegt.

In [44]:
! kubectl config view -o=jsonpath='{ .clusters[0].cluster.server }' | sed -e 's/https:/http:/' -e "s/6443/$(kubectl get service --namespace test apache -o=jsonpath='{ .spec.ports[0].nodePort }')/"


http://192.168.178.200:32243

Zum Aufräumen genügt es den Namespace zu löschen

In [45]:
! kubectl delete namespace test

namespace "test" deleted
